# Elaboration d'un graphe en profondeur

Choix de hubs subjectifs de départ :  Le Figaro, qui est de "droite gaulliste, libérale et conservatrice".  Libération, qui est "un journal libertaire de gauche"

1. Initialisation de Tweepy

In [1]:
import tweepy

In [2]:
from tweepy import OAuthHandler

In [3]:
consumer_key = 'L2mPv8Oxjs2mLtBEWZZiyDy0W'
consumer_secret = 'idPHQogjNLSKdo7ns8exk9ZXSbIsqcd4UsXd1vofliwj1s2iZj'
access_token = '1413867470-bWJRTSz2Ece90tG1Z5IU9pmPZQvdWR10xrWqNBL'
access_secret = '6Z0X9eJ0dLlYJj8JrqX6VcNMOOsTWpaxaFosJhiv5Xkoa'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

2. Initialisation des hubs

In [4]:
hub1 = api.get_user('Le_Figaro')
hub2 = api.get_user('libe')
print(hub1.screen_name)
print(hub1.followers_count)
print('-----------')
print(hub2.screen_name)
print(hub2.followers_count)

Le_Figaro
3340630
-----------
libe
3182480


3. Récupération de quelques comptes au hasard parmi les 600 000 (pas besoin de tous les prendre, la limite de twitter va nous limiter à ne parcourir les abonnements que de quelques centaines d'utilisateurs)

In [5]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

In [8]:
import pandas as pd 
import numpy as np
import csv

In [73]:
followersids_hub1 = []
followersids_hub2 = []

In [74]:
with open("followersid_hub1.csv", newline="") as f: 
  csvreader = csv.reader(f) 
  for row in csvreader: 
    i = int(row[0])
    followersids_hub1.append(i)
with open("followersid_hub2.csv", newline="") as f: 
  csvreader = csv.reader(f) 
  for row in csvreader: 
    i = int(row[0])
    followersids_hub2.append(i)

In [95]:
users_hub1 = []
for i in range(50):            
    fin_cycle = min((i + 1) * 50, len(followersids_hub1))
    users_hub1.extend(
        api.lookup_users(user_ids=followersids_hub1[i * 50:fin_cycle])                
    )

In [96]:
users_hub2 = []
for i in range(50):            
    fin_cycle = min((i + 1) * 50, len(followersids_hub2))
    try:
        users_hub2.extend(
            api.lookup_users(user_ids=followersids_hub2[i * 50:fin_cycle])                
        )
    except:
        print('User suivant')

In [97]:
# On retire les comptes doublons
print(len(users_hub1))
print(len(users_hub2))
for j in users_hub1:
    for i in users_hub2:
        if i.screen_name == j.screen_name:
            users_hub1.remove(i)
print(len(users_hub1))
print(len(users_hub2))

2493
2494
1821
2494


4. Récupération des abonnements de 340 followers des hubs (170 hub1, 170 hub2). Environ 4h30 de temps. 

In [99]:
abonnementsids_hub1 = {}
for i in range(170):
    # Pour chaque utilisateur, on récupère ses abonnements avec la même méthode
    tmpids = []
    # Ne pas oublier la gestion d'erreur pour les comptes privés (jusque 13%), il y en a beaucoup !!
    try:
        for abonnement in tweepy.Cursor(api.friends_ids, screen_name=users_hub1[i].screen_name, count=5000).items():
            tmpids.append(abonnement)
    except tweepy.TweepError: 
        print("Erreur utilisateur, au suivant")
    abonnementsids_hub1[users_hub1[i].screen_name] = tmpids
len(abonnementsids_hub1)

Rate limit reached. Sleeping for: 479


Erreur utilisateur, au suivant


Rate limit reached. Sleeping for: 893


Erreur utilisateur, au suivant
Erreur utilisateur, au suivant


Rate limit reached. Sleeping for: 893


...

Rate limit reached. Sleeping for: 893


Erreur utilisateur, au suivant
Erreur utilisateur, au suivant


Rate limit reached. Sleeping for: 892


Erreur utilisateur, au suivant


170

In [100]:
abonnementsids_hub2 = {}
for i in range(170):
    # Pour chaque utilisateur, on récupère ses abonnements avec la même méthode
    tmpids = []
    # Ne pas oublier la gestion d'erreur pour les comptes privés (jusque 13%), il y en a beaucoup !!
    try:
        for abonnement in tweepy.Cursor(api.friends_ids, screen_name=users_hub2[i].screen_name, count=5000).items():
            tmpids.append(abonnement)
    except tweepy.TweepError: 
        print("Erreur utilisateur, au suivant")
    abonnementsids_hub2[users_hub2[i].screen_name] = tmpids
len(abonnementsids_hub2)

Erreur utilisateur, au suivant


Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892


Erreur utilisateur, au suivant


Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892


Erreur utilisateur, au suivant
Erreur utilisateur, au suivant


...

Rate limit reached. Sleeping for: 892


Erreur utilisateur, au suivant


Rate limit reached. Sleeping for: 892


Erreur utilisateur, au suivant


170

In [101]:
print(len(abonnementsids_hub1))
print(len(abonnementsids_hub2))

170
170


In [102]:
# Nombre d'abonnement que l'on a dans les 300
res = 0
for k, v in abonnementsids_hub1.items():
    for j in range(len(v)):
        res = res + 1
for k, v in abonnementsids_hub2.items():
    for j in range(len(v)):
        res = res + 1
print(res)

54986


In [103]:
df1 = pd.DataFrame.from_dict(abonnementsids_hub1, orient ='index')
df1.to_csv('abonnementsids_hub1.csv', sep =',')
df2 = pd.DataFrame.from_dict(abonnementsids_hub2, orient ='index')
df2.to_csv('abonnementsids_hub2.csv', sep =',')

5. Partie algorithmique : déterminer les liens entre ces abonnements pour définir les hubs

In [104]:
abonnementsids_final = {}
for key, value in abonnementsids_hub1.items():
        if len(value) > 0:
            abonnementsids_final[key] = value;
print(len(abonnementsids_hub1))
print(len(abonnementsids_final))

res = 0
for k, v in abonnementsids_final.items():
    for j in range(len(v)):
        res = res + 1
print(res)

abonnementsids_final_2 = {}
for key, value in abonnementsids_hub2.items():
        if len(value) > 0:
            abonnementsids_final_2[key] = value;
print(len(abonnementsids_hub2))
print(len(abonnementsids_final_2))

res = 0
for k, v in abonnementsids_final_2.items():
    for j in range(len(v)):
        res = res + 1
print(res)

170
153
23919
170
158
31067


In [105]:
def existe(cle, liste):
    if cle in liste:
        return "true"
    return "false"

In [106]:
liens = {}
liste_visite_ids = []
for k, v in abonnementsids_final.items():
    nbLiens = 0
    for i in range(len(v)):
        for key, value in abonnementsids_final.items():
            for j in range(len(value)):
                if(v[i] == value[j] and k != key and existe(key, liste_visite_ids) == "false"):
                    nbLiens = nbLiens + 1
        if(existe(v[i], liens) == "true"):
            liens[v[i]] += nbLiens
        else:
            liens[v[i]] = nbLiens
    liste_visite_ids.append(k)      
print(len(liens))

liens_2 = {}
liste_visite_ids_2 = []
for k, v in abonnementsids_final_2.items():
    nbLiens = 0
    for i in range(len(v)):
        for key, value in abonnementsids_final_2.items():
            for j in range(len(value)):
                if(v[i] == value[j] and k != key and existe(key, liste_visite_ids_2) == "false"):
                    nbLiens = nbLiens + 1
        if(existe(v[i], liens_2) == "true"):
            liens_2[v[i]] += nbLiens
        else:
            liens_2[v[i]] = nbLiens
    liste_visite_ids_2.append(k)      
print(len(liens_2))

17413
24026


Pour bien comprendre : on a les abonnements de plus de 300 utilisateurs, en tout 54 986 personnes. On dit que un lien, c'est un abonnement. Donc on compte le nombre de fois où un utilisateur apparait. On a alors 17 413 liens pour le hub 1, et 24 026 liens pour le hub 2.

In [134]:
# On tri les liens
liensTries = {}
liensTries_2 = {}
for k, v in sorted(liens.items(), key=lambda x: x[1], reverse=True):
    liensTries[k] = v
for k, v in sorted(liens_2.items(), key=lambda x: x[1], reverse=True):
    liensTries_2[k] = v

In [138]:
usersid_tries = []
usersid_tries_2 = []
lien_pondere = []
lien_pondere_2 = []
for k, v in liensTries.items():
    usersid_tries.append(k)
    lien_pondere.append(v)
for k, v in liensTries_2.items():
    usersid_tries_2.append(k)
    lien_pondere_2.append(v)

In [139]:
hubs = []
for i in range(int(len(usersid_tries)/100) + 1):            
    fin_cycle = min((i + 1) * 100, len(usersid_tries))
    hubs.extend(
        api.lookup_users(user_ids=usersid_tries[i * 100:fin_cycle])                
    )

In [140]:
hubs_2 = []
for i in range(int(len(usersid_tries_2)/100) + 1):            
    fin_cycle = min((i + 1) * 100, len(usersid_tries_2))
    hubs_2.extend(
        api.lookup_users(user_ids=usersid_tries_2[i * 100:fin_cycle])                
    )

In [141]:
# Concernant la communauté de Figaro
for i in range(len(hubs)):
    print("Nb liens : ", lien_pondere[i], " est ", hubs[i].screen_name, " avec ", hubs[i].followers_count, " followers.")

Nb liens :  28983  est  EmmanuelMacron  avec  6405104  followers.
Nb liens :  28827  est  lemondefr  avec  9050772  followers.
Nb liens :  25285  est  Le_Figaro  avec  3340987  followers.
Nb liens :  21993  est  BarackObama  avec  126672313  followers.
Nb liens :  19043  est  JoeBiden  avec  19827537  followers.
Nb liens :  18175  est  le_Parisien  avec  2696324  followers.
Nb liens :  16872  est  RFI  avec  2678762  followers.
Nb liens :  16680  est  BFMTV  avec  2947381  followers.
Nb liens :  14854  est  France24_fr  avec  3594734  followers.
Nb liens :  14754  est  CNEWS  avec  1724464  followers.
Nb liens :  14476  est  franceinfo  avec  1753190  followers.
Nb liens :  12932  est  jeune_afrique  avec  3053251  followers.
Nb liens :  12925  est  afpfr  avec  3571883  followers.
Nb liens :  12146  est  realDonaldTrump  avec  88886695  followers.
Nb liens :  11874  est  NetflixFR  avec  3966438  followers.
Nb liens :  11342  est  Elysee  avec  2632321  followers.
Nb liens :  11077  e

Nb liens :  2490  est  andresilverkona  avec  48193  followers.
Nb liens :  2467  est  TPMP  avec  3500014  followers.
Nb liens :  2465  est  SecCivileFrance  avec  52434  followers.
Nb liens :  2464  est  LEFAauthentique  avec  183177  followers.
Nb liens :  2463  est  Koaci  avec  156321  followers.
Nb liens :  2462  est  laurentwauquiez  avec  337683  followers.
Nb liens :  2446  est  CotedIvoire_off  avec  158451  followers.
Nb liens :  2443  est  MIdrissDebyItno  avec  71381  followers.
Nb liens :  2442  est  lusakuenoc  avec  166152  followers.
Nb liens :  2432  est  DeniseNyakeru  avec  167891  followers.
Nb liens :  2431  est  vonderleyen  avec  433402  followers.
Nb liens :  2426  est  JM_Aulas  avec  605041  followers.
Nb liens :  2418  est  saleh_kebzabo  avec  26667  followers.
Nb liens :  2405  est  AgnesRunacher  avec  39992  followers.
Nb liens :  2389  est  BBCAfrica  avec  3150388  followers.
Nb liens :  2380  est  Lacrim_Officiel  avec  1491202  followers.
Nb liens : 

Nb liens :  1827  est  dydy_iam  avec  2762  followers.
Nb liens :  1827  est  dunnysea  avec  89988  followers.
Nb liens :  1827  est  LezouW  avec  50  followers.
Nb liens :  1827  est  DomJDom  avec  11587  followers.
Nb liens :  1827  est  RosytaEnriquez  avec  168  followers.
Nb liens :  1827  est  giuliabottini  avec  3748  followers.
Nb liens :  1827  est  FerraresiPV  avec  1480  followers.
Nb liens :  1826  est  narendramodi  avec  63774758  followers.
Nb liens :  1821  est  ManCity  avec  8731687  followers.
Nb liens :  1804  est  AshleyLelaMAIGA  avec  30012  followers.
Nb liens :  1804  est  konate90  avec  45776  followers.
Nb liens :  1802  est  JeanLucLemoine  avec  1885921  followers.
Nb liens :  1795  est  PrimatureRDC  avec  277110  followers.
Nb liens :  1793  est  LFC  avec  15897853  followers.
Nb liens :  1793  est  AldoSterone111  avec  48885  followers.
Nb liens :  1789  est  el_pais  avec  7844051  followers.
Nb liens :  1788  est  atmabri  avec  20527  followe

Nb liens :  1418  est  AdriendeVriendt  avec  153  followers.
Nb liens :  1418  est  saranda_nc  avec  19  followers.
Nb liens :  1418  est  andreaskaplan  avec  786  followers.
Nb liens :  1418  est  edouardhusson  avec  5050  followers.
Nb liens :  1418  est  Motion_Life  avec  30  followers.
Nb liens :  1418  est  matthieu_pichon  avec  126  followers.
Nb liens :  1418  est  CecileDuflot  avec  406826  followers.
Nb liens :  1418  est  ClaireBalva  avec  5751  followers.
Nb liens :  1418  est  escp_europe  avec  4466  followers.
Nb liens :  1418  est  RenaudCamus  avec  47489  followers.
Nb liens :  1417  est  FranceOTAN  avec  11897  followers.
Nb liens :  1416  est  RaphaelleBacque  avec  184686  followers.
Nb liens :  1416  est  AllhassaneSimon  avec  56079  followers.
Nb liens :  1415  est  ArianeChemin  avec  73566  followers.
Nb liens :  1414  est  PlayStation  avec  20379170  followers.
Nb liens :  1413  est  lettegubert  avec  5728  followers.
Nb liens :  1411  est  Frederic

Nb liens :  1185  est  Albadrawi6  avec  1470  followers.
Nb liens :  1185  est  Mbodouliwa  avec  110  followers.
Nb liens :  1185  est  non_yal  avec  595  followers.
Nb liens :  1185  est  CommelinMoussa  avec  904  followers.
Nb liens :  1185  est  idriss_maide  avec  441  followers.
Nb liens :  1185  est  AvocksoumaProf  avec  982  followers.
Nb liens :  1185  est  CJAC_TD  avec  1386  followers.
Nb liens :  1185  est  _KhalidDaoud  avec  699  followers.
Nb liens :  1185  est  MediaToubou  avec  2016  followers.
Nb liens :  1185  est  DanayeSouleyman  avec  572  followers.
Nb liens :  1185  est  fatihtufekcipa1  avec  265  followers.
Nb liens :  1185  est  mhat_2006  avec  81  followers.
Nb liens :  1185  est  tchadhanana  avec  2439  followers.
Nb liens :  1185  est  SahoulbaE  avec  12296  followers.
Nb liens :  1185  est  Soukaya_Chemi  avec  212  followers.
Nb liens :  1185  est  Moudwe  avec  1552  followers.
Nb liens :  1185  est  pedrofirtion  avec  2911  followers.
Nb lien

...

Nb liens :  6  est  nicomoramunoz  avec  307  followers.
Nb liens :  6  est  seb_otero  avec  1442  followers.
Nb liens :  6  est  CamaraNacional  avec  7050  followers.
Nb liens :  6  est  Alvaro_Escobar_  avec  132656  followers.
Nb liens :  6  est  DiarioConce  avec  51962  followers.
Nb liens :  6  est  Chumisj  avec  8310  followers.
Nb liens :  6  est  EmbaChileEspana  avec  764  followers.
Nb liens :  6  est  FundChileEspana  avec  996  followers.
Nb liens :  6  est  apec2021NZ  avec  1055  followers.
Nb liens :  6  est  rohanamahmood13  avec  100  followers.
Nb liens :  6  est  barbarabricenok  avec  66389  followers.
Nb liens :  6  est  LiamFox  avec  100601  followers.
Nb liens :  6  est  trussliz  avec  91997  followers.
Nb liens :  6  est  Isabel_Ramos_DF  avec  1261  followers.
Nb liens :  6  est  INTALBID  avec  10616  followers.
Nb liens :  6  est  Princeton  avec  419504  followers.
Nb liens :  6  est  Yale  avec  545507  followers.
Nb liens :  6  est  stephanygj  avec 

Nb liens :  6  est  ScottMorrisonMP  avec  489756  followers.
Nb liens :  6  est  raimundolarrain  avec  4452  followers.
Nb liens :  6  est  ChileConference  avec  239  followers.
Nb liens :  6  est  Contraloriacl  avec  125199  followers.
Nb liens :  6  est  DFinanciero  avec  211949  followers.
Nb liens :  6  est  udeconcepcion  avec  28930  followers.
Nb liens :  6  est  Ciencia_UdeC  avec  3818  followers.
Nb liens :  6  est  PACYTbiobio  avec  2093  followers.
Nb liens :  6  est  mjabud  avec  260  followers.
Nb liens :  6  est  caccgp  avec  530  followers.
Nb liens :  6  est  FPiensa  avec  18142  followers.
Nb liens :  6  est  Rebecca_APEC  avec  1691  followers.
Nb liens :  6  est  IISS_org  avec  136316  followers.
Nb liens :  6  est  CFR_org  avec  473065  followers.
Nb liens :  6  est  ChathamHouse  avec  218351  followers.
Nb liens :  6  est  hans_ebeni  avec  4659  followers.
Nb liens :  6  est  gdelafue  avec  22544  followers.
Nb liens :  6  est  rpalaciosp  avec  331 

Nb liens :  4  est  Nikaa_Dww  avec  208930  followers.
Nb liens :  4  est  cyrilschreiner  avec  481452  followers.
Nb liens :  4  est  PraxITT  avec  303  followers.
Nb liens :  4  est  nesyouELK  avec  42370  followers.
Nb liens :  4  est  MemesDeSpies  avec  27999  followers.
Nb liens :  4  est  karmaaOfficiel  avec  246664  followers.
Nb liens :  4  est  idkbvtimleila  avec  330921  followers.
Nb liens :  4  est  victorperez_v  avec  7260  followers.
Nb liens :  4  est  GalerieduJour  avec  5421  followers.
Nb liens :  3  est  OfficielGalerie  avec  10876  followers.
Nb liens :  3  est  DowntonAbbey  avec  631358  followers.
Nb liens :  3  est  nonprofitcoffee  avec  150704  followers.
Nb liens :  3  est  OEWisdom  avec  6049  followers.
Nb liens :  3  est  LRB  avec  294425  followers.
Nb liens :  3  est  ForeignPolicy  avec  1155108  followers.
Nb liens :  3  est  IntlCrimCourt  avec  465973  followers.
Nb liens :  3  est  OMSMaroc  avec  16091  followers.
Nb liens :  3  est  Re

Nb liens :  0  est  yasminesabri  avec  2377533  followers.
Nb liens :  0  est  KindaAlloush  avec  1635112  followers.
Nb liens :  0  est  nadinenjeim  avec  1093017  followers.
Nb liens :  0  est  assihallani  avec  3551982  followers.
Nb liens :  0  est  sherine  avec  7032188  followers.
Nb liens :  0  est  Jesus_Le_Vrai1  avec  23  followers.
Nb liens :  0  est  PitaudManon31  avec  121  followers.
Nb liens :  0  est  martinpatry16  avec  62  followers.
Nb liens :  0  est  TitouanLEC_pro  avec  271  followers.
Nb liens :  0  est  _Thomas_Andre  avec  83  followers.
Nb liens :  0  est  etujournalistes  avec  645  followers.
Nb liens :  0  est  MahamadouDaoud3  avec  204  followers.
Nb liens :  0  est  Magori227  avec  251  followers.
Nb liens :  0  est  LBakasso  avec  135  followers.
Nb liens :  0  est  zodisidi1  avec  60  followers.
Nb liens :  0  est  SalimZanguina  avec  1066  followers.
Nb liens :  0  est  ahametelhrabe  avec  121  followers.
Nb liens :  0  est  AuthenticSnip

Nb liens :  0  est  RaiScuola  avec  30748  followers.
Nb liens :  0  est  ZackMwekassa  avec  16506  followers.
Nb liens :  0  est  InaGelbert  avec  8142  followers.
Nb liens :  0  est  GTA_Rockstar_FR  avec  3464  followers.
Nb liens :  0  est  IfokuMarijo  avec  26712  followers.
Nb liens :  0  est  bakongajonathan  avec  526  followers.
Nb liens :  0  est  TonnyMpungu  avec  181  followers.
Nb liens :  0  est  MONUSCO  avec  381842  followers.
Nb liens :  0  est  FTViaStella  avec  52587  followers.
Nb liens :  0  est  alwasatnewsly  avec  183279  followers.
Nb liens :  0  est  Ez3Yz4j3JNh1dWE  avec  129  followers.
Nb liens :  0  est  Love_YSL_PRgirl  avec  14425  followers.
Nb liens :  0  est  YSL  avec  4121962  followers.
Nb liens :  0  est  THR  avec  3269369  followers.
Nb liens :  0  est  gcr_alerts  avec  3313  followers.
Nb liens :  0  est  Concurrences  avec  2391  followers.
Nb liens :  0  est  IsabelleDeSilva  avec  2577  followers.
Nb liens :  0  est  Adlc_  avec  736

In [142]:
# Concernant la communauté de Libération
for i in range(len(hubs_2)):
    print("Nb liens : ", lien_pondere_2[i], " est ", hubs_2[i].screen_name, " avec ", hubs_2[i].followers_count, " followers.")

Nb liens :  39645  est  lemondefr  avec  9050778  followers.
Nb liens :  33825  est  EmmanuelMacron  avec  6405123  followers.
Nb liens :  31699  est  libe  avec  3182954  followers.
Nb liens :  28109  est  BarackObama  avec  126672429  followers.
Nb liens :  24371  est  le_Parisien  avec  2696324  followers.
Nb liens :  19995  est  franceinfo  avec  1753195  followers.
Nb liens :  19590  est  BFMTV  avec  2947384  followers.
Nb liens :  19481  est  JoeBiden  avec  19828095  followers.
Nb liens :  17914  est  CNEWS  avec  1724470  followers.
Nb liens :  17744  est  afpfr  avec  3571884  followers.
Nb liens :  16819  est  Le_Figaro  avec  3340987  followers.
Nb liens :  16436  est  Mediapart  avec  2675621  followers.
Nb liens :  16328  est  CNN  avec  51144731  followers.
Nb liens :  15767  est  RFI  avec  2678767  followers.
Nb liens :  15655  est  France24_fr  avec  3594740  followers.
Nb liens :  15358  est  KamalaHarris  avec  12341317  followers.
Nb liens :  14491  est  NetflixFR 

Nb liens :  2716  est  la_Bible  avec  431538  followers.
Nb liens :  2715  est  leroidesrats  avec  275678  followers.
Nb liens :  2709  est  JLB_officiel  avec  93212  followers.
Nb liens :  2700  est  AJEnglish  avec  6514188  followers.
Nb liens :  2691  est  Nice_Matin  avec  211015  followers.
Nb liens :  2677  est  Independent  avec  3442722  followers.
Nb liens :  2672  est  AFP  avec  1945394  followers.
Nb liens :  2663  est  MileyCyrus  avec  45483120  followers.
Nb liens :  2653  est  Aubameyang7  avec  1719812  followers.
Nb liens :  2641  est  tweetsamoureux  avec  2370484  followers.
Nb liens :  2630  est  Kulturlesite_  avec  368259  followers.
Nb liens :  2619  est  Space_Station  avec  4039174  followers.
Nb liens :  2604  est  INCIndia  avec  7372854  followers.
Nb liens :  2602  est  iledefrance  avec  86080  followers.
Nb liens :  2601  est  TV5MONDEAfrique  avec  167901  followers.
Nb liens :  2597  est  LuisSuarez9  avec  16602487  followers.
Nb liens :  2594  es

Nb liens :  2067  est  WFP  avec  1803816  followers.
Nb liens :  2067  est  VigiMeteoFrance  avec  55278  followers.
Nb liens :  2058  est  le_brouteur  avec  140820  followers.
Nb liens :  2055  est  _AfricanUnion  avec  599735  followers.
Nb liens :  2053  est  booska_p  avec  594182  followers.
Nb liens :  2048  est  TheDemocrats  avec  2000337  followers.
Nb liens :  2045  est  le_gorafi  avec  1321713  followers.
Nb liens :  2030  est  espn  avec  36140277  followers.
Nb liens :  2019  est  LIDER_CI  avec  58561  followers.
Nb liens :  2016  est  M_Koulibaly  avec  153210  followers.
Nb liens :  2015  est  DeptofDefense  avec  6283963  followers.
Nb liens :  2013  est  yasminaouegnin  avec  81329  followers.
Nb liens :  2010  est  PapeSidy  avec  91178  followers.
Nb liens :  2008  est  CanalFootClub  avec  1356517  followers.
Nb liens :  2000  est  10Ronaldinho  avec  19785741  followers.
Nb liens :  2000  est  Footballogue  avec  1235048  followers.
Nb liens :  1996  est  Sonko

Nb liens :  1444  est  adgpi  avec  7005299  followers.
Nb liens :  1443  est  SpokespersonMoD  avec  443088  followers.
Nb liens :  1442  est  indiannavy  avec  374279  followers.
Nb liens :  1441  est  GKS_TV  avec  54961  followers.
Nb liens :  1441  est  SebLecornu  avec  56553  followers.
Nb liens :  1439  est  JeudyBruno  avec  445106  followers.
Nb liens :  1439  est  xavierbertrand  avec  178405  followers.
Nb liens :  1438  est  FinancialTimes  avec  6796906  followers.
Nb liens :  1435  est  HenonStephane  avec  13997  followers.
Nb liens :  1435  est  FABIENNE_CARAT  avec  66559  followers.
Nb liens :  1435  est  douniacoesens  avec  53503  followers.
Nb liens :  1435  est  leafrancoisoff  avec  62337  followers.
Nb liens :  1435  est  PBLVofficiel  avec  231820  followers.
Nb liens :  1435  est  LaurentKerusore  avec  50932  followers.
Nb liens :  1435  est  LMilotOfficiel  avec  187088  followers.
Nb liens :  1435  est  gaak_fr  avec  16588  followers.
Nb liens :  1435  es

Nb liens :  1322  est  OliverOtwist  avec  155  followers.
Nb liens :  1322  est  Ibrahimfkeita  avec  307  followers.
Nb liens :  1322  est  mthy_k  avec  35943  followers.
Nb liens :  1322  est  AbdoulALenkone  avec  292  followers.
Nb liens :  1322  est  papillonnoir224  avec  12345  followers.
Nb liens :  1322  est  ModiboCheickD  avec  2229  followers.
Nb liens :  1322  est  Djelmatw  avec  77416  followers.
Nb liens :  1322  est  malikberi12  avec  6159  followers.
Nb liens :  1322  est  AlhassaneSid1  avec  1265  followers.
Nb liens :  1322  est  liyanatur  avec  3610  followers.
Nb liens :  1322  est  Modibotoure90  avec  4525  followers.
Nb liens :  1322  est  SeydinaOumar28  avec  1666  followers.
Nb liens :  1322  est  Touitteur2  avec  16322  followers.
Nb liens :  1322  est  Nando_Influent  avec  20888  followers.
Nb liens :  1322  est  Tonton_Py  avec  2917  followers.
Nb liens :  1322  est  Konibasanogo0  avec  76  followers.
Nb liens :  1322  est  Kadidjatou47  avec  80

...

Nb liens :  2  est  SAWNEYY_  avec  4172  followers.
Nb liens :  2  est  mxha_db  avec  1027  followers.
Nb liens :  2  est  followback_95  avec  1390  followers.
Nb liens :  2  est  VEO_KiNg  avec  1320  followers.
Nb liens :  2  est  legrandpeet  avec  634  followers.
Nb liens :  2  est  musica50100p  avec  1702  followers.
Nb liens :  2  est  followback  avec  132676  followers.
Nb liens :  2  est  ReviewSearchFR  avec  7077  followers.
Nb liens :  2  est  100_FollowBackx  avec  9762  followers.
Nb liens :  2  est  _Celiaa_m  avec  8184  followers.
Nb liens :  2  est  Hz7iyech_  avec  2428  followers.
Nb liens :  2  est  souley_silue  avec  3456  followers.
Nb liens :  2  est  Salmaaabr  avec  2907  followers.
Nb liens :  2  est  Thierno27269606  avec  1879  followers.
Nb liens :  2  est  danielleebassa  avec  420  followers.
Nb liens :  2  est  Ihsane____  avec  1520  followers.
Nb liens :  2  est  MZognie  avec  254  followers.
Nb liens :  2  est  svnaah93  avec  1493  followers.


Nb liens :  1  est  SSMBEMPIRE  avec  6037  followers.
Nb liens :  1  est  GMBents  avec  59572  followers.
Nb liens :  1  est  14ReelsPlus  avec  12822  followers.
Nb liens :  1  est  Akhiles35244871  avec  15  followers.
Nb liens :  1  est  kedianomics  avec  29557  followers.
Nb liens :  1  est  RiuHoteles  avec  243566  followers.
Nb liens :  1  est  Sauravk1802  avec  213  followers.
Nb liens :  1  est  AshokNParmar1  avec  34  followers.
Nb liens :  1  est  RichardEngel  avec  434871  followers.
Nb liens :  1  est  RamBarajas3  avec  3869  followers.
Nb liens :  1  est  PierreHerme_FR  avec  12911  followers.
Nb liens :  1  est  cyril_lignac  avec  365313  followers.
Nb liens :  1  est  MORIHIDEYOSHIDA  avec  1336  followers.
Nb liens :  1  est  GuideRestos  avec  283  followers.
Nb liens :  1  est  mori_yoshida  avec  198  followers.
Nb liens :  1  est  boetmie  avec  24  followers.
Nb liens :  1  est  TheNewPainting  avec  252953  followers.
Nb liens :  1  est  peac4love  avec 

Nb liens :  0  est  LDFloveyou  avec  123  followers.
Nb liens :  0  est  holdup_ledoc  avec  13796  followers.
Nb liens :  0  est  _Gilets_Jaunes_  avec  39301  followers.
Nb liens :  0  est  GiletsJaunesFr_  avec  17454  followers.
Nb liens :  0  est  GJaunes  avec  10453  followers.
Nb liens :  0  est  IkerCasillas  avec  9309176  followers.
Nb liens :  0  est  SpinsLinks  avec  34008  followers.
Nb liens :  0  est  daily_spins  avec  14762  followers.
Nb liens :  0  est  CoinMasterGame  avec  884379  followers.
Nb liens :  0  est  CestMonChoixOff  avec  29586  followers.
Nb liens :  0  est  RMCStory  avec  22786  followers.
Nb liens :  0  est  Cherie25tv  avec  19013  followers.
Nb liens :  0  est  6ter  avec  36402  followers.
Nb liens :  0  est  CStar  avec  255852  followers.
Nb liens :  0  est  borisbachorz  avec  3085  followers.
Nb liens :  0  est  jeunes_francean  avec  74  followers.
Nb liens :  0  est  AvenirEduc  avec  268  followers.
Nb liens :  0  est  JeunesMacron69  a

Nb liens :  0  est  Jay_RMA  avec  7511  followers.
Nb liens :  0  est  FCBarcelonaFl  avec  173980  followers.
Nb liens :  0  est  CrimiPunch  avec  338  followers.
Nb liens :  0  est  ponchovpof  avec  104228  followers.
Nb liens :  0  est  LongHaired_One  avec  5085  followers.
Nb liens :  0  est  katbuble  avec  1592  followers.
Nb liens :  0  est  MamaDJ77  avec  451  followers.
Nb liens :  0  est  Nope_No_way_FU  avec  8526  followers.
Nb liens :  0  est  kittyAnanya  avec  480584  followers.
Nb liens :  0  est  sluttyx1  avec  1951078  followers.
Nb liens :  0  est  sexhijabhyper  avec  1380  followers.
Nb liens :  0  est  bibiayesha00  avec  8447  followers.
Nb liens :  0  est  MuslimGirlx__  avec  4326  followers.
Nb liens :  0  est  otarww  avec  80128  followers.
Nb liens :  0  est  67V007y4ughqH37  avec  8125  followers.
Nb liens :  0  est  muthrer4  avec  55757  followers.
Nb liens :  0  est  couples_mot7arr  avec  166905  followers.
Nb liens :  0  est  OM_LO_88  avec  156

Nb liens :  0  est  daltrofilho  avec  46  followers.
Nb liens :  0  est  breda_luciane  avec  557  followers.
Nb liens :  0  est  MarceloChiavass  avec  73  followers.
Nb liens :  0  est  civcatt  avec  36162  followers.
Nb liens :  0  est  CNBBNacional  avec  96570  followers.
Nb liens :  0  est  vaticannews_pt  avec  78532  followers.
Nb liens :  0  est  fam_cristiana  avec  50330  followers.
Nb liens :  0  est  Avvenire_Nei  avec  84724  followers.
Nb liens :  0  est  fattoquotidiano  avec  2028880  followers.
Nb liens :  0  est  CorSport  avec  716384  followers.
Nb liens :  0  est  yabamongo  avec  8799  followers.
Nb liens :  0  est  KubraYacubMust  avec  4541  followers.
Nb liens :  0  est  AzouraDiaz  avec  6016  followers.
Nb liens :  0  est  AlamarMamary1  avec  4624  followers.
Nb liens :  0  est  IdHakima  avec  6374  followers.
Nb liens :  0  est  SahoulbaE  avec  12296  followers.
Nb liens :  0  est  LEauEnPoudre_  avec  1743  followers.
Nb liens :  0  est  tetanos235  a

On créé les csv nécessaires pour le graphe

In [143]:
final_dico_figaro = {}
final_dico_libe = {}
for i in range(len(hubs)):
    final_dico_figaro[hubs[i].screen_name] = lien_pondere[i]
for i in range(len(hubs_2)):
    final_dico_libe[hubs_2[i].screen_name] = lien_pondere_2[i]

In [144]:
df3 = pd.DataFrame.from_dict(final_dico_figaro, orient ='index')
df3.to_csv('final_dico_figaro.csv', sep =',')
df4 = pd.DataFrame.from_dict(final_dico_libe, orient ='index')
df4.to_csv('final_dico_libe.csv', sep =',')

In [148]:
final_dico_unifie = {} 
for d in final_dico_figaro, final_dico_libe: 
    for key in d: 
        final_dico_unifie[key] = final_dico_unifie.get(key, 0) + d[key] 
len(final_dico_unifie)

38342

In [150]:
df_final = pd.DataFrame.from_dict(final_dico_unifie, orient ='index')
df_final.to_csv('dico_final.csv', sep =',')

In [164]:
dico_figaro_user_abonnements = {}
for k, v in abonnementsids_final.items():
    new_t = []
    for i in range(int(len(v)/100) + 1):
        fin_cycle = min((i + 1) * 100, len(v))
        new_t.extend(
            api.lookup_users(user_ids=v[i * 100:fin_cycle])                
        )
    dico_figaro_user_abonnements[k] = new_t

In [170]:
for k, v in dico_figaro_user_abonnements.items():
    nom = ''
    for i in range(len(v)):
        nom = v[i].screen_name
        v[i] = nom

In [173]:
dico_libe_user_abonnements = {}
for k, v in abonnementsids_final_2.items():
    new_t = []
    for i in range(int(len(v)/100) + 1):
        fin_cycle = min((i + 1) * 100, len(v))
        new_t.extend(
            api.lookup_users(user_ids=v[i * 100:fin_cycle])                
        )
    dico_libe_user_abonnements[k] = new_t

for k, v in dico_libe_user_abonnements.items():
    nom = ''
    for i in range(len(v)):
        nom = v[i].screen_name
        v[i] = nom

In [171]:
df_final_figaro_abonnements = pd.DataFrame.from_dict(dico_figaro_user_abonnements, orient ='index')
df_final_figaro_abonnements.to_csv('dico_user_abos_fig.csv', sep =',')

In [174]:
df_final_libe_abonnements = pd.DataFrame.from_dict(dico_libe_user_abonnements, orient ='index')
df_final_libe_abonnements.to_csv('dico_user_abos_libe.csv', sep =',')